In [1]:
import gurobipy as gb
from gurobipy import *
import numpy as np

# Rummikub

Rummikub is a game that combines elements of classic card games like Rummy with the strategy of tile placement. The game is played with a set of 106 tiles, with numbers ranging from 1 to 13 in four different colors (red, blue, yellow, and black). Additionally, there are two joker tiles in the set. The game is typically played by 2 to 4 players and revolves around the strategic placement and manipulation of numbered tiles. The objective of Rummikub is to be the first player to empty your rack of tiles by forming sets and runs of matching numbers. Sets consist of three or four tiles of the same number but different colors. For example, you could have a set of 3s with one red, one blue, and one black. Runs are sequences of at least three consecutive numbers of the same color. For instance, you could have a run of 4, 5, 6 in blue. The game continues until one player goes out, at which point they gain opponents' tile values, while others receive penalties determined by the remaining tiles in their racks.

In this project, our primary objective is to address Rummikub challenges through the application of integer linear programming. Initially, we plan to focus on a two-player scenario, with the potential to expand to a four-player format if time permits. Also, if time allows, our ultimate goal is to develop an interactive online Rummikub board game, providing users with a platform for engaging gameplay.

## Set up
2 users version: 4*13 + 2 joker

In [2]:
# Deck = {"Black": ["1", "1", "2", "2", "3", "3", "4", "4", "5", "5", "6", "6", 
#                   "7", "7", "8", "8", "9", "9", "10", "10", "11", "11", "12", "12", "13", "13"], 
#         "Red": ["1", "1", "2", "2", "3", "3", "4", "4", "5", "5", "6", "6", 
#                   "7", "7", "8", "8", "9", "9", "10", "10", "11", "11", "12", "12", "13", "13"], 
#         "Blue": ["1", "1", "2", "2", "3", "3", "4", "4", "5", "5", "6", "6", 
#                   "7", "7", "8", "8", "9", "9", "10", "10", "11", "11", "12", "12", "13", "13"], 
#         "Orange": ["1", "1", "2", "2", "3", "3", "4", "4", "5", "5", "6", "6", 
#                   "7", "7", "8", "8", "9", "9", "10", "10", "11", "11", "12", "12", "13", "13"], 
#         "Joker": ["J1", "J2"]}

In [2]:
Penalty = {"1": 1, "2":2, "3":3, "4":4, "5":5, "6":6, "7":7, "8":8, "9": 9, "10":10, "11":11, "12":12, 
           "13":13, "Joker": 30}

In [9]:
Value = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
Value_matrix = Value * 4

In [4]:
model = gb.Model("Rummikub")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-30


## Decision variables

In [5]:
# Deck_types = ["Black1", "Black2", "Red1", "Red2", "Blue1", "Blue2", "Orange1", "Orange2"]
Deck_types = ["Black", "Red", "Orange", "Blue"]
Deck_values = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13"]
Joker = ["Joker"]
Joker_values = ["J1", "J2"]


Deck1 = model.addVars(Deck_types, Deck_values, vtype = GRB.BINARY, name = "Tile1")
# Deck2 = model.addVars(Deck_types, Deck_values, vtype = GRB.BINARY, name = "Tile2")
Joker1 = model.addVars(Joker, Joker_values, vtype = GRB.BINARY, name = "Tile1_Joker")
# Joker2 = model.addVars(Joker, Joker_values, vtype = GRB.BINARY, name = "Tile2_Joker")

In [6]:
I = len(Value_matrix)
J = 1174

In [7]:
S_reg = model.addVars(Deck_types, Deck_values, J, vtype = GRB.BINARY, 
                  name = ["tile "+ Deck_type + Deck_value + " is in set " + str(j+1) 
                          for Deck_type in Deck_types for Deck_value in Deck_values for j in range(J)])
S_jok = model.addVars(Joker, Joker_values, J, vtype = GRB.BINARY, 
                      name = ["tile Joker " + jok + " is in set " + str(j+1) for jok in Joker_values for j in range(J)])

T_reg = model.addVars(Deck_types, Deck_values, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                  name = ["times of tile " + Deck_type + Deck_value + " on the table" 
                          for Deck_type in Deck_types for Deck_value in Deck_values])
T_jok = model.addVars(Joker, Joker_values, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                  name = ["times of tile Joker" + jok +"on the table" for jok in Joker_values])

R_reg = model.addVars(Deck_types, Deck_values, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                   name = ["times of tile "+ Deck_type + Deck_value +" on your rack" 
                           for Deck_type in Deck_types for Deck_value in Deck_values])
R_jok = model.addVars(Joker, Joker_values, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                  name = ["times of tile Joker" + jok +"on the table" for jok in Joker_values])

X_reg = model.addVars(J, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                  name = ["times of set "+str(j+1)+" can be placed onto the table" for j in range(J)])

Y_reg = model.addVars(Deck_types, Deck_values, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                  name = ["times of tile "+ Deck_type + Deck_value +" can be placed from your rack onto the table" 
                          for Deck_type in Deck_types for Deck_value in Deck_values])
Y_jok = model.addVars(Joker, Joker_values, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                  name = ["times of tile Joker" + jok +" can be placed from your rack onto the table" for jok in Joker_values])
# W = model.addVars(J, lb = 0, ub = 2, vtype = GRB.INTEGER, 
#                   name = ["times of set "+str(j+1)+" on the table" for j in range(J)])
# M = 40
# Z = model.addVars(J, lb = 0, ub = 2, vtype = GRB.INTEGER, 
#                   name = ["times of set "+str(j+1)+" occurs in the old and in the new solution" for j in range(J)])

In [8]:
Y_reg

{('Black', '1'): <gurobi.Var *Awaiting Model Update*>,
 ('Black', '2'): <gurobi.Var *Awaiting Model Update*>,
 ('Black', '3'): <gurobi.Var *Awaiting Model Update*>,
 ('Black', '4'): <gurobi.Var *Awaiting Model Update*>,
 ('Black', '5'): <gurobi.Var *Awaiting Model Update*>,
 ('Black', '6'): <gurobi.Var *Awaiting Model Update*>,
 ('Black', '7'): <gurobi.Var *Awaiting Model Update*>,
 ('Black', '8'): <gurobi.Var *Awaiting Model Update*>,
 ('Black', '9'): <gurobi.Var *Awaiting Model Update*>,
 ('Black', '10'): <gurobi.Var *Awaiting Model Update*>,
 ('Black', '11'): <gurobi.Var *Awaiting Model Update*>,
 ('Black', '12'): <gurobi.Var *Awaiting Model Update*>,
 ('Black', '13'): <gurobi.Var *Awaiting Model Update*>,
 ('Red', '1'): <gurobi.Var *Awaiting Model Update*>,
 ('Red', '2'): <gurobi.Var *Awaiting Model Update*>,
 ('Red', '3'): <gurobi.Var *Awaiting Model Update*>,
 ('Red', '4'): <gurobi.Var *Awaiting Model Update*>,
 ('Red', '5'): <gurobi.Var *Awaiting Model Update*>,
 ('Red', '6'): <

## Objective function

In [9]:
# model.setObjective(quicksum(Value[i] * Y[i] for i in range(I)) + 1/M * quicksum(Z[j] for j in range(J)), GRB.MAXIMIZE)
exp1 = quicksum(Value_matrix[i] * Y_reg[i] for i in range(I))
exp2 = quicksum(Y_jok[i] for i in range(2)) * 30
model.setObjective(exp1 + exp2, GRB.MAXIMIZE)

## Constraints

In [10]:
for i in range(I):
    model.addConstr(quicksum(S[i, j]*X[j] for j in range(J)) == T[i] + Y[i])
#model.addConstrs(quicksum(S[i, j]*X[j] for j in range(J)) == T[i] + Y[i] for i in range(I))

In [11]:
model.addConstrs(Y[i] <= R[i] for i in range(I))
model.addConstrs(Z[j] <= X[j] for j in range(J))
model.addConstrs(Z[j] <= W[j] for j in range(J))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

## Optimize

In [12]:
model.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2401 rows, 65903 columns and 4802 nonzeros
Model fingerprint: 0xa388fad1
Model has 53 quadratic constraints
Variable types: 0 continuous, 65903 integer (62222 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [3e-02, 3e+01]
  Bounds range     [1e+00, 2e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 2401 rows and 2401 columns
Presolve time: 0.15s
Presolved: 186719 rows, 125724 columns, 497882 nonzeros
Variable types: 0 continuous, 125724 integer (62222 binary)

Root relaxation: objective 8.467000e+02, 0 iterations, 0.07 seconds (0.08 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work


In [13]:
model.ObjVal

846.6999999999466

In [16]:
# for v in model.getVars():
#     if v.x != 0:
#         print (v.varName, v.x)

##### Some thoughts

decision variables maybe should be separated by color

interactive so that have enough tile on the table to solve and optimize

after solving using ILP, show the solution in plain language, also show how many extra steps it will need to finish the game